In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pickle
os.chdir('../')
os.chdir('scripts')
import data_creation_v3 as d

In [10]:
os.chdir('../')
os.chdir('models')

In [11]:
order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
       'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
       'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
       'sscr', 'urlIsLive', 'urlLength']

In [13]:
a = d.UrlFeaturizer('http://astore.amazon.co.uk/allezvinsfrenchr/detail/1904010202/026-8324244-9330038').run()
test = []
for i in order:
    test.append(a[i])
encoder = LabelEncoder()
encoder.classes_ = np.load('lblenc.npy',allow_pickle=True)
scalerfile = 'scaler.sav'
scaler = pickle.load(open(scalerfile, 'rb'))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.21.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [22]:
test = pd.DataFrame(test).replace(True,1).replace(False,0).to_numpy(dtype="float32").reshape(1,-1)
test = scaler.transform(test)

In [24]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite_quant_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], test)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted = np.argmax(output_data,axis=1)

In [25]:
print(encoder.inverse_transform(predicted)[0])

Malware_dataset
